In [191]:
import copy
import pandas as pd
import numpy as np
import librosa
import seaborn as sb
import matplotlib.pyplot as plt
import itertools
import re
import random
import gc
import math
import os
from sklearn.utils import shuffle
from operator import itemgetter, attrgetter, methodcaller
from os import listdir
from os.path import isfile, join
from numpy import median, diff
from xgboost import XGBClassifier
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout, BatchNormalization
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.externals import joblib
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from mlxtend.classifier import EnsembleVoteClassifier

### TODOS
- generate percent of single double notes etc with a nn

## Classes
- 1: one note
- 2: two notes
- 3: three or four notes
- 4: hold start
- 5: roll start
- 6: mine

In [147]:
def get_class_for_index_expanded(notes, index):
    if index < 0:
        return [1, 0, 0, 0, 0, 0, 0]
    row = notes[index][0]
    (steps, holds, rolls, mines) = [row.count(char) for char in ['1', '2', '4', 'M']]
    if steps == 0 and mines == 0 and holds == 0 and rolls == 0:
        return [1, 0, 0, 0, 0, 0, 0]
    steps += (holds + rolls)
    return [int(i) for i in [False, steps == 1, steps == 2, steps > 2, holds > 0, rolls > 0, mines > 0]]

def get_class_for_index(notes, index):
    classes_expanded = get_class_for_index_expanded(notes, index)
    return [i for i in range(7) if classes_expanded[i]]

In [131]:
steps_per_bar = 48
class SongFile:
    def __init__(self, key):
        misc = pd.read_csv('data/{0}_misc.csv'.format(key)).values
        self.bpm = misc[1][0]
        self.notes = pd.read_csv('data/{0}_notes.csv'.format(key), converters={'0': lambda x: str(x)}).values
        self.note_classes = [get_class_for_index_expanded(self.notes, i) for i in range(len(self.notes))]

In [121]:
beats_to_track = 48
note_types = ['0', '1', 'M', '2', '4', '3']

def get_features_for_row(row):
    return [int(char == target) for target in note_types for char in row]

empty_row = [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
def get_previous_notes(index, features):
    previous_notes = [features[i] for i in range(index, index + song_padding) if not np.array_equal(features[i], empty_row)]
    return [empty_row] * (8 - len(previous_notes)) + previous_notes[-8:]
    
song_padding = beats_to_track * 2
song_end_padding = beats_to_track * 2
important_indices = [1, 2, 3, 4, 8, 16, 20, 6, 12, 18, 24, 30, 36, 42, 48, 54, 60, 66, 72, 78, 84, 90, 96]
important_indices_classes = [-96, -84, -72, -60, -48, -36, -24, -12, 0, 1, 2, 3, 4, 8, 16, 20, 6, 12, 18, 24, 30, 36, 42, 48, 54, 60, 66, 72, 78, 84, 90, 96]
def get_features(index, features, note_classes):
    indices = [index + song_padding - i for i in important_indices]
    indices_classes = [index + song_padding - i for i in important_indices_classes]
    past_classes = np.array([note_classes[i] for i in indices_classes]).flatten()
    past_features = np.array([features[i] for i in indices]).flatten()
    previous_notes = np.array(get_previous_notes(index, features)).flatten()
    return np.concatenate((past_classes, past_features, previous_notes), axis = 0)

In [122]:
def get_model_output_for_class(model_class, row):
    if model_class == 1 or model_class == 2 or model_class == 3:
        return [int(char == '1' or char == '2' or char == '4') for char in row]
    if model_class == 4:
        return [int(char == '2') for char in row]
    if model_class == 5:
        return [int(char == '4') for char in row]
    if model_class == 6:
        return [int(char == 'M') for char in row]

def get_hold_length(notes, note_row, note_column):
    i = 0
    while i < len(notes) - note_row:
        if notes[note_row + i][0][note_column] == '3':
            return i
        i += 1
    return False

In [150]:
def get_features_for_songs(songs):
    hold_X = []
    roll_X = []
    hold_y = []
    roll_y = []
    X = [[] for i in range(6)]
    y = [[] for i in range(6)]
    for song in songs:
        note_classes = np.concatenate((([[1, 0, 0, 0, 0, 0, 0]] * song_padding), song.note_classes, ([[1, 0, 0, 0, 0, 0, 0]] * song_end_padding)), axis = 0)
        notes = np.concatenate((([['0000']] * song_padding), song.notes), axis = 0)
        if abs(len(note_classes) - len(notes) > 250):
            print ('Lengths dont match for {0}'.format(key))
            print ('{0} vs {1}'.format(len(note_classes), len(notes)))
            continue
        length = min(len(note_classes) - song_padding - song_end_padding, len(notes) - song_padding)
        features = np.array([get_features_for_row(notes[i][0]) for i in range(0, length + song_padding)])
        for i in range(length):
            row = notes[i + song_padding][0]
            model_classes = get_class_for_index(notes, i + song_padding)
            for model_class in model_classes:
                X_row = get_features(i, features, note_classes)
                X[model_class].append(X_row)
                y[model_class].append(get_model_output_for_class(model_class, row))
                if model_class == 4:
                    for j in range(4):
                        if row[j] == '2':
                            length = get_hold_length(notes, i + song_padding, j)
                            if length:
                                hold_X.append(X_row)
                                hold_y.append(length)
                if model_class == 5:
                    for j in range(4):
                        if row[j] == '4':
                            length = get_hold_length(notes, i + song_padding, j)
                            if length:
                                roll_X.append(X_row)
                                roll_y.append(length)

    X = [np.array(X_for_class) for X_for_class in X]
    y = [np.array(y_for_class) for y_for_class in y]
    return X, y, np.array(hold_X), np.array(hold_y), np.array(roll_X), np.array(roll_y)

In [151]:
songs_to_use_full = pd.read_csv('data/songs_to_use.csv').values
save_files = listdir('data')
songs_to_use = [song_data for song_data in songs_to_use_full if '{0}_misc.csv'.format(song_data[0]) in save_files]
songs = [SongFile(song_data[0]) for song_data in songs_to_use]
np.random.shuffle(songs)

In [153]:
%%time
X_train_array, y_train_array, hold_X_train, hold_y_train, roll_X_train, roll_y_train = get_features_for_songs(songs[:174]) # total 217
X_test_array, y_test_array, hold_X_test, hold_y_test, roll_X_test, roll_y_test = get_features_for_songs(songs[174:])

CPU times: user 10min 48s, sys: 14 s, total: 11min 2s
Wall time: 11min 8s


In [156]:
hold_lengths = [3, 6, 9, 12, 18, 24, 36, 48]
def get_closest_hold_lengths(lengths):
    closest = [np.argmax([-abs(length - aprox) for aprox in hold_lengths]) for length in lengths]
    closest_one_hot = np.zeros((len(closest), len(hold_lengths)))
    closest_one_hot[np.arange(len(closest)), closest] = 1
    return np.array(closest_one_hot)

In [157]:
hold_y_train_backup_array = hold_y_train_array
roll_y_train_backup_array = roll_y_train_array
hold_y_test_backup_array = hold_y_test_array
roll_y_test_backup_array = roll_y_test_array
hold_y_train_array = get_closest_hold_lengths(hold_y_train_backup_array)
roll_y_train_array = get_closest_hold_lengths(roll_y_train_backup_array)
hold_y_test_array = get_closest_hold_lengths(hold_y_test_backup_array)
roll_y_test_array = get_closest_hold_lengths(roll_y_test_backup_array)

In [161]:
def build_model(num_classes):
    model = Sequential()

    model.add(Dense(1024, input_shape=(968,)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.2))

    model.add(Dense(1024))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.2))

    model.add(Dense(num_classes))
    model.add(BatchNormalization())
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='adagrad',
                  metrics=['accuracy'])
    
    return model

In [159]:
model = build_model(len(hold_lengths))
model.fit(hold_X_train, hold_y_train, nb_epoch=5, batch_size=64, verbose=1, validation_data=(hold_X_test, hold_y_test))
model.save('models/hold_length_model.h5')

Train on 6977 samples, validate on 1637 samples
Epoch 1/5
6977/6977 [==============================] - 2s - loss: 1.7676 - acc: 0.3841 - val_loss: 1.8668 - val_acc: 0.3158
Epoch 2/5
6977/6977 [==============================] - 2s - loss: 1.4505 - acc: 0.5199 - val_loss: 1.7549 - val_acc: 0.3965
Epoch 3/5
6977/6977 [==============================] - 2s - loss: 1.3247 - acc: 0.5759 - val_loss: 1.6736 - val_acc: 0.4264
Epoch 4/5
6977/6977 [==============================] - 2s - loss: 1.2196 - acc: 0.6219 - val_loss: 1.6354 - val_acc: 0.4386
Epoch 5/5
6977/6977 [==============================] - 2s - loss: 1.1336 - acc: 0.6531 - val_loss: 1.6109 - val_acc: 0.4417


In [164]:
model = build_model(len(hold_lengths))
model.fit(roll_X_train, roll_y_train, nb_epoch=5, batch_size=64, verbose=1, validation_data=(roll_X_test, roll_y_test))
model.save('models/roll_length_model.h5')

Train on 192 samples, validate on 70 samples
Epoch 1/5
192/192 [==============================] - 0s - loss: 1.9776 - acc: 0.2552 - val_loss: 2.2029 - val_acc: 0.1714
Epoch 2/5
192/192 [==============================] - 0s - loss: 1.1365 - acc: 0.7240 - val_loss: 2.0339 - val_acc: 0.1857
Epoch 3/5
192/192 [==============================] - 0s - loss: 0.8839 - acc: 0.8698 - val_loss: 1.9465 - val_acc: 0.2714
Epoch 4/5
192/192 [==============================] - 0s - loss: 0.7811 - acc: 0.9010 - val_loss: 1.9388 - val_acc: 0.2429
Epoch 5/5
192/192 [==============================] - 0s - loss: 0.6839 - acc: 0.9531 - val_loss: 1.9223 - val_acc: 0.2143


In [188]:
%%time
batch_sizes = [0, 64, 16, 2, 16, 4, 8]
models = []
models.append(None)
for X_train, y_train, X_test, y_test in zip():
    model = build_model(4)
    model.fit(X_train[i], y_train[i], nb_epoch=5, batch_size=batch_sizes[i], verbose=1, validation_data=(X_test[i], y_test[i]))
    model.save('models/note_model_{0}.h5'.format(i))
    models.append(model)

Train on 78876 samples, validate on 18412 samples
Epoch 1/5
78876/78876 [==============================] - 58s - loss: 1.1993 - acc: 0.4197 - val_loss: 1.1624 - val_acc: 0.4378
Epoch 2/5
78876/78876 [==============================] - 54s - loss: 1.1287 - acc: 0.4699 - val_loss: 1.1536 - val_acc: 0.4430
Epoch 3/5
78876/78876 [==============================] - 57s - loss: 1.0828 - acc: 0.5023 - val_loss: 1.1605 - val_acc: 0.4426
Epoch 4/5
78876/78876 [==============================] - 58s - loss: 1.0371 - acc: 0.5342 - val_loss: 1.1845 - val_acc: 0.4369
Epoch 5/5
78876/78876 [==============================] - 57s - loss: 0.9931 - acc: 0.5596 - val_loss: 1.1808 - val_acc: 0.4390
Train on 6031 samples, validate on 1412 samples
Epoch 1/5
6031/6031 [==============================] - 14s - loss: 2.7893 - acc: 0.2742 - val_loss: 2.7565 - val_acc: 0.2075
Epoch 2/5
6031/6031 [==============================] - 13s - loss: 2.6909 - acc: 0.3203 - val_loss: 2.7774 - val_acc: 0.2323
Epoch 3/5
6031/60

In [260]:
class_arrays = [
    [],
    ['1000', '0100', '0010', '0001'],
    ['1100', '1010', '1001', '0110', '0101', '0011'],
    ['1110', '1101', '1011', '0111', '1111'],
    ['1000', '0100', '0010', '0001', '2', '3', '4'],
    ['1000', '0100', '0010', '0001', '2', '3', '4'],
    ['1000', '0100', '0010', '0001', '2', '3', '4'],
]
class_maps = [dict((class_array[i], i) for i in range(len(class_array))) for class_array in class_arrays]
def get_class(class_map, y_row):
    as_string = ''.join(str(x) for x in y_row)
    pos_count = as_string.count('1')
    return class_map[str(pos_count)] if '2' in class_map and pos_count > 1 else class_map[as_string]

def get_y_not_one_hot(y):
    return [[get_class(class_map, y_row) for y_row in y_section] for class_map, y_section in zip(class_maps[1:], y[1:])]

In [261]:
y_train_classes = get_y_not_one_hot(y_train)
y_test_classes = get_y_not_one_hot(y_test)

In [264]:
%%time
min_samples_leafs = [0, 32, 8, 1, 8, 2, 4]
rfs = []
rfs.append(None)
for train, train_y, test, test_y in zip(X_train[1:], y_train_classes, X_test[1:], y_test_classes):
    rf_clf = RandomForestClassifier(n_estimators = 50) #, min_samples_leaf=min_samples_leafs[i])
    rf_clf.fit(train, train_y)
    print (rf_clf.score(train, train_y))
    print (rf_clf.score(test, test_y))
    rfs.append(rf_clf)

0.999188599828
0.451499022377
1.0
0.235127478754
1.0
0.25
1.0
0.425359150531
1.0
0.285714285714
1.0
0.339226519337
CPU times: user 48.5 s, sys: 2.54 s, total: 51.1 s
Wall time: 54.7 s


In [189]:
%%time
sgds = []
sgds.append(None)
for i in range(0, 6):
    sgd_clf = SGDClassifier(loss="modified_huber", n_iter=10)
    sgd_clf.fit(X_train[i], y_train_classes[i])
    print (sgd_clf.score(X_train[i], y_train_classes[i]))
    print (sgd_clf.score(X_test[i], y_test_classes[i]))
    sgds.append(sgd_clf)

ValueError: bad input shape (78876, 4)

In [192]:
%%time
xgbs = []
xgbs.append(None)
for i in range(0, 6):
    xgb_clf = XGBClassifier(max_depth=7, min_child_weight=8, learning_rate=0.05, seed=0, n_estimators=100, subsample=0.80, colsample_bytree=0.80, objective="multi:softprob")
    xgb_clf.fit(X_train[i], y_train_classes[i])
    print (xgb_clf.score(X_train[i], y_train_classes[i]))
    print (xgb_clf.score(X_test[i], y_test_classes[i]))
    xgbs.append(xgb_clf)

ValueError: bad input shape (78876, 4)